# Q1: Scalable Serverless Architectures

### Step0: Set up all dependencies

In [1]:
import json
import boto3
import os

### Step1: Define the send_survey function

In [2]:
def send_survey(survey_path, sqs_url):
    '''
        Input:
            survey_path (str): path to JSON survey data (e.g. ‘./survey.json’)
            sqs_url (str): URL for SQS queue
        Output:
            StatusCode (int): indicating whether the survey was successfully 
            sent into the SQS queue (200) or not (0)
    '''
    # Load survey data from the file
    with open(survey_path, 'r') as file:
        survey_data = json.load(file)

    # Create a boto3 client for SQS
    sqs_client = boto3.client('sqs')

    # Send the message to the SQS queue
    try:
        response = sqs_client.send_message(
            QueueUrl=sqs_url,
            MessageBody=json.dumps(survey_data)
        )
        return 200
    except Exception as e:
        print(f"Failed to send message: {e}")
        return 0

### Step2: Create S3 Bucket & DynamoDB Table

In [3]:
lambda_client = boto3.client('lambda')
iam_client = boto3.client('iam')

# Initialize S3 client
s3_client = boto3.client('s3')
# Define the bucket name
bucket_name = 'a2q1-bucket'

# Create the S3 bucket
try:
    s3_client.create_bucket(Bucket=bucket_name)
    print(f"S3 bucket {bucket_name} created successfully.")
except Exception as e:
    print(f"Error creating S3 bucket {bucket_name}: {str(e)}")

# Attempt to retrieve the IAM role
try:
    role = iam_client.get_role(RoleName='LabRole')
    role_arn = role['Role']['Arn']
except Exception as e:
    print(f"Error retrieving IAM role: {str(e)}")

# Initialize a DynamoDB client
dynamodb = boto3.resource('dynamodb')
dynamodb_client = boto3.client('dynamodb')
# Define the table name
table_name = 'a2q1-table'

# Create the DynamoDB table
try: 
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {'AttributeName': 'user_id', 'KeyType': 'HASH'},
            {'AttributeName': 'timestamp', 'KeyType': 'RANGE'}
        ],
        AttributeDefinitions=[
            {'AttributeName': 'user_id', 'AttributeType': 'S'},
            {'AttributeName': 'timestamp', 'AttributeType': 'S'}
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 1,
            'WriteCapacityUnits': 1
        }
    )
    table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
    print(f"DynamoDB table {table_name} created successfully.")
except Exception as e:
    print(f"Error creating DynamoDB table: {str(e)}")


S3 bucket a2q1-bucket created successfully.
DynamoDB table a2q1-table created successfully.


### Step3: Create Lambda function

In [7]:
with open('q1c_lambda.zip', 'rb') as f:
    lambda_zip = f.read()

# Create or update Lambda function
try:
    response = lambda_client.create_function(
        FunctionName='a2q1_lambda',
        Runtime='python3.11',
        Role=role['Role']['Arn'],
        Handler='q1c_lambda.lambda_handler',
        Code=dict(ZipFile=lambda_zip),
        Timeout=300
)
    print(response)
except lambda_client.exceptions.ResourceConflictException:
    response = lambda_client.update_function_code(
        FunctionName='a2q1_lambda',
        ZipFile=lambda_zip
        )
    print(response)
except Exception as e:
    print(f"Error creating or updating Lambda function: {str(e)}")

# Set concurrency for Lambda function
try:
    response = lambda_client.put_function_concurrency(
        FunctionName='a2q1_lambda',
        ReservedConcurrentExecutions=10
    )
    print(response)
except Exception as e:
    print(f"Error setting concurrency for Lambda function: {str(e)}")

{'ResponseMetadata': {'RequestId': 'b34a034e-8f5c-43a3-81b1-e5e2ca64f69a', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Thu, 02 May 2024 19:42:07 GMT', 'content-type': 'application/json', 'content-length': '1309', 'connection': 'keep-alive', 'x-amzn-requestid': 'b34a034e-8f5c-43a3-81b1-e5e2ca64f69a'}, 'RetryAttempts': 0}, 'FunctionName': 'a2q1_lambda', 'FunctionArn': 'arn:aws:lambda:us-east-1:102168828713:function:a2q1_lambda', 'Runtime': 'python3.11', 'Role': 'arn:aws:iam::102168828713:role/LabRole', 'Handler': 'q1c_lambda.lambda_handler', 'CodeSize': 1208, 'Description': '', 'Timeout': 300, 'MemorySize': 128, 'LastModified': '2024-05-02T19:42:07.749+0000', 'CodeSha256': 'm7oerYXAOsyLz55Wm0oPPhfabWdEtKhv/2gMDWfisCc=', 'Version': '$LATEST', 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': '8558eb6f-1398-4589-b658-7caf622da4e0', 'State': 'Pending', 'StateReason': 'The function is being created.', 'StateReasonCode': 'Creating', 'PackageType': 'Zip', 'Architectures': ['x86_64'],

### Step4: Create SQS & Configure SQS to act as a trigger for a Lambda function

In [9]:
import boto3

# Create an SQS client
sqs_client = boto3.client('sqs')
lambda_client = boto3.client('lambda')  # Ensure lambda_client is defined
# Define the name of the queue
queue_name = 'a2q1-queue'

# Create the queue
try:
    response = sqs_client.create_queue(
        QueueName=queue_name
    )
    print("Queue created successfully:", response['QueueUrl'])
except sqs_client.exceptions.QueueNameExists:
    print("Queue with the same name already exists.")
except Exception as e:
    print("An error occurred:", e)

# Set visibility timeout and create event source mapping
try:
    # Get the URL of the queue
    queue_url_response = sqs_client.get_queue_url(QueueName=queue_name)
    queue_url = queue_url_response['QueueUrl']

    # Get the queue ARN using the queue URL
    queue_attributes_response = sqs_client.get_queue_attributes(
        QueueUrl=queue_url,
        AttributeNames=['QueueArn']
    )
    queue_arn = queue_attributes_response['Attributes']['QueueArn']

    # Set queue attributes
    sqs_client.set_queue_attributes(
        QueueUrl=queue_url,
        Attributes={'VisibilityTimeout': '1800'}
    )
    print("Visibility timeout set successfully.")

    # Create event source mapping between Lambda and the SQS queue
    response = lambda_client.create_event_source_mapping(
        FunctionName='a2q1_lambda',
        EventSourceArn=queue_arn,
        Enabled=True,
        BatchSize=10
    )
    mapping_uuid = response['UUID']
    print("Event source mapping created successfully:", response)

except Exception as e:
    print(f"An error occurred: {str(e)}")

Queue created successfully: https://sqs.us-east-1.amazonaws.com/102168828713/a2q1-queue
Visibility timeout set successfully.
Event source mapping created successfully: {'ResponseMetadata': {'RequestId': '8a18f498-15e9-49c1-a19b-e76066f0344c', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Thu, 02 May 2024 19:43:01 GMT', 'content-type': 'application/json', 'content-length': '875', 'connection': 'keep-alive', 'x-amzn-requestid': '8a18f498-15e9-49c1-a19b-e76066f0344c'}, 'RetryAttempts': 0}, 'UUID': 'c2d71cfc-da22-495a-acea-eec4da949b81', 'BatchSize': 10, 'MaximumBatchingWindowInSeconds': 0, 'EventSourceArn': 'arn:aws:sqs:us-east-1:102168828713:a2q1-queue', 'FunctionArn': 'arn:aws:lambda:us-east-1:102168828713:function:a2q1_lambda', 'LastModified': datetime.datetime(2024, 5, 2, 14, 43, 1, 186000, tzinfo=tzlocal()), 'State': 'Creating', 'StateTransitionReason': 'USER_INITIATED', 'FunctionResponseTypes': []}


### Testing: Send .json test data to the queue

In [10]:
json_directory = './test_json'

# Check if the directory exists
if not os.path.exists(json_directory):
    print(f"Directory not found: {json_directory}")
else:
    # Iterate over each file in the directory and send it to the SQS queue
    for filename in os.listdir(json_directory):
        if filename.endswith('.json'):
            file_path = os.path.join(json_directory, filename)
            status_code = send_survey(file_path, queue_url)
            print(f"File: {filename}, Status Code: {status_code}")

File: test1.json, Status Code: 200
File: test6.json, Status Code: 200
File: test7.json, Status Code: 200
File: test8.json, Status Code: 200
File: test4.json, Status Code: 200
File: test5.json, Status Code: 200
File: test9.json, Status Code: 200
File: test10.json, Status Code: 200
File: test2.json, Status Code: 200
File: test3.json, Status Code: 200


### Ending: Delete relevant services

In [11]:
# Delete the event source mapping
try:
    lambda_client.delete_event_source_mapping(UUID=mapping_uuid)
    print("Event source mapping deleted successfully")
except Exception as e:
    print("Failed to delete event source mapping:", str(e))
    
# Delete the SQS queue
try:
    sqs_response = sqs_client.delete_queue(QueueUrl=queue_url)
    print("SQS Queue deleted successfully:", sqs_response)
except Exception as e:
    print("Failed to delete SQS Queue:", str(e))

# Delete the S3 bucket and all its contents
try:
    bucket_objects = s3_client.list_objects_v2(Bucket=bucket_name)
    if 'Contents' in bucket_objects:
        for obj in bucket_objects['Contents']:
            s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
    s3_client.delete_bucket(Bucket=bucket_name)
    print("S3 Bucket deleted successfully")
except Exception as e:
    print("Failed to delete S3 Bucket:", str(e))

# Delete the DynamoDB table
try:
    dynamodb_client.delete_table(TableName=table_name)
    print("DynamoDB Table deleted successfully")
except Exception as e:
    print("Failed to delete DynamoDB Table:", str(e))

# Delete the Lambda function
try:
    lambda_client.delete_function(FunctionName='a2q1_lambda')
    print("Lambda Function deleted successfully")
except Exception as e:
    print("Failed to delete Lambda Function:", str(e))



Event source mapping deleted successfully
SQS Queue deleted successfully: {'ResponseMetadata': {'RequestId': '88530b10-42ac-5e8e-a8e5-55bdadfc82e8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '88530b10-42ac-5e8e-a8e5-55bdadfc82e8', 'date': 'Thu, 02 May 2024 19:44:16 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '0', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
S3 Bucket deleted successfully
DynamoDB Table deleted successfully
Lambda Function deleted successfully
